In [1]:
import plotly.express as px
import pandas as pd
from google.cloud import bigquery
import numpy as np
from datetime import time, timedelta, datetime
import math 
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets
import statsmodels.stats.weightstats as sw
from statsmodels.stats.proportion import test_proportions_2indep
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
from google.cloud import bigquery
from scipy.stats import norm
from scipy import stats
pd.options.display.float_format = "{:,.2f}".format
from scipy.stats import norm
from scipy import stats
import statsmodels.stats.weightstats as ws
import openpyxl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

# df["Week"] = df["Date"].dt.to_period("W").dt.to_timestamp()

bigquery_client = bigquery.Client(project='analytics-dev-333113')

def cycle_sql(start, end, query, weeks=False):
    """
    You have to use {date} in your script to add cycle date into this backets
    """
    date_start = datetime.strptime(start, '%Y-%m-%d')
    date_end = datetime.strptime(end, '%Y-%m-%d')

    if weeks == False:
        daterange = [(date_start + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days)+1)]
    else:
        daterange = [(date_start + timedelta(weeks=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days//7)+1)] # weeks dividing days by 7

    total_df = pd.DataFrame()

    for date in daterange:
        print(f"Uploading - {date}:", datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
        script = query.format(date = date)
        df_cycle = bigquery_client.query(script).to_dataframe()
        if df_cycle.empty == True:
            print('Dataframe is empty')
        total_df = pd.concat([df_cycle, total_df])
    return total_df

def read_bq(query, project='analytics-dev-333113'):
    client = bigquery.Client(project=project)
    query_job = client.query(query)
    result_df = query_job.to_dataframe()
    return result_df

def writing_excel(name:str, dataset1=None, dataset2=None, dataset3=None, dataset4=None):
    with pd.ExcelWriter(f"{name}.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet

        if dataset1 is not None:
            if dataset2 is not None:
                if dataset3 is not None:
                    if dataset4 is not None:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                        dataset4.to_excel(writer, sheet_name=f"4-{name}", 
                                        #   index=False
                                          )
                    else:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                else:
                    dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                    #   index=False
                                      )
                    dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                    #   index=False
                                      )
            else:
                dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                #   index=False
                                  )

        print('DataFrame is written to Excel File successfully.')


/Users/renatyunison/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Number of active users since June 2024 (Active - who has at least 1 ride per period)

In [2]:
active_users = read_bq("""
WITH cte AS (SELECT user_type,
                    user_id,
                    t1.country_id,
                    country_name,
                    t1.city_id,
                    city_name,
                    t2.macroregion_name,
                    metric_date,
                    rides_count,
                    IF(rides_count > 0, 1, 0)                                                                             AS has_ride,
                    LEAD(metric_date) OVER (PARTITION BY user_id ORDER BY metric_date)                                    AS next_ride_date,
                    SUM(rides_count)
                        OVER (PARTITION BY user_id ORDER BY metric_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_sum
             FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
                      JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                           ON t1.city_id = t2.city_id
             WHERE metric_date >= '2024-07-01'
               AND IF(rides_count > 0, 1, 0) = 1
               AND user_type = 'pass')
SELECT DATE_TRUNC(metric_date, MONTH) AS metric_date_monthly,
       country_name,
       macroregion_name,
       COUNT(DISTINCT user_id)        AS users
FROM cte
GROUP BY 1, 2, 3
""")

active_users.head()

,metric_date_monthly,country_name,macroregion_name,users
0,2024-10-01,USA,Developed countries,22349
1,2024-10-01,Nigeria,Africa,337004
2,2024-10-01,Costa Rica,Latin America,128804
3,2024-10-01,Algeria,MENA,227767
4,2024-11-01,Казахстан,CIS(СНГ),1263772


In [11]:
active_users.groupby(['metric_date_monthly', 'country_name', 'macroregion_name'], as_index=False)['users'].agg('sum')

,metric_date_monthly,country_name,macroregion_name,users
0,2024-07-01,Algeria,MENA,188268
1,2024-07-01,Argentina,Latin America,58393
2,2024-07-01,Bangladesh,SA,32544
3,2024-07-01,Bolivia,Latin America,311599
4,2024-07-01,Botswana,Africa,37242
...,...,...,...,...
259,2024-11-01,Zambia,Africa,1977
260,2024-11-01,Zimbabwe,Africa,65476
261,2024-11-01,Казахстан,CIS(СНГ),1263772
262,2024-11-01,Кыргызстан,CIS(СНГ),16665


In [8]:
active_users.pivot_table(values='users', index='metric_date_monthly',
                       columns='macroregion_name', aggfunc="sum")

macroregion_name,Africa,Brazil,CIS(СНГ),Developed countries,Latin America,MENA,SA,SEA&O
metric_date_monthly,,,,,,,,
2024-07-01,860731,1644345,2243392,16888,10845555,3012353,4326181,2009319
2024-08-01,923975,1697232,2337725,19766,11082076,3135395,4450599,2015695
2024-09-01,954662,1721160,2155014,21700,11051627,3008938,4398327,2081652
2024-10-01,1014871,1734681,2239488,22682,11227492,2938863,4548859,2121093
2024-11-01,588078,1021472,1328412,12322,7052292,1546691,2455734,1281656


# New users since June 2024 (Everyone who provided OTP in registration step)

In [3]:
new_users = read_bq("""
WITH cte AS (SELECT t1.id,
                    CAST(t1.created AS date)      AS reg_date,
                    DATE_TRUNC(t1.created, MONTH) AS month,
                    t1.mode,
                    t1.country_id,
                    t2.country_name,
                    t2.macroregion_name,
                    t1.city_id,
                    t2.city_name,
                    CASE
                        WHEN osversion LIKE '%android%' THEN 'android'
                        WHEN osversion LIKE '%ios%' THEN 'ios'
                        ELSE
                            NULL
                        END
                                                     os
             FROM indriver-e6e40.ods_monolith.tbl_user t1
                      JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                           ON t1.city_id = t2.city_id
             WHERE 1 = 1
               AND CAST(t1.created AS DATE) >= '2024-07-01'
               AND t1.mode = 'client'
             QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.id ORDER BY t1.created DESC) = 1)
SELECT DATE(reg_date)                    AS created_daily,
       DATE_TRUNC(DATE(reg_date), MONTH) AS created_monthly,
       country_name,
       macroregion_name,
       os,
       COUNT(DISTINCT id)                AS users
FROM cte
GROUP BY 1, 2, 3, 4, 5
ORDER BY users DESC
""")

new_users.head()

,created_daily,created_monthly,country_name,macroregion_name,os,users
0,2024-08-19,2024-08-01,India,SA,android,31749
1,2024-11-03,2024-11-01,India,SA,android,22305
2,2024-07-07,2024-07-01,India,SA,android,21379
3,2024-07-14,2024-07-01,India,SA,android,21361
4,2024-07-06,2024-07-01,India,SA,android,21028


In [12]:
new_users.pivot_table(values='users', index='created_monthly',
                       columns='macroregion_name', aggfunc="sum")

macroregion_name,Africa,Brazil,CIS(СНГ),China,Developed countries,Europe,Latin America,MENA,SA,SEA&O
created_monthly,,,,,,,,,,
2024-07-01,289275,374627,183693,997,51048,4416,1223810,907965,1188447,493971
2024-08-01,346353,348347,174900,1366,58194,4674,1192946,895463,1089193,476828
2024-09-01,341758,352615,129988,964,48535,4093,1170753,698703,996702,530573
2024-10-01,346713,368972,146772,1058,46090,3519,1143229,655552,1107637,536614
2024-11-01,126800,114237,48694,198,15275,989,390185,209333,362687,195954


# Shield ID. Distinct number & Reccuring IDs & Average number by users

In [33]:
shield_id = cycle_sql('2024-10-01', '2024-10-10',"""
SELECT DISTINCT fl.user_id,
                sf.platform,
                fl.event_dt_part,
                sf.session_dt_part,
                sf.shield_id,
--                 sf.session_dt_part,
--                 ROW_NUMBER() OVER (PARTITION BY fl.user_id) AS duplicates
FROM indriver-e6e40.antifraud.fingerprint_log fl
         JOIN indriver-e6e40.antifraud.shield_fingerprints sf USING (fingerprint_id)
WHERE event_dt_part = '{date}'
  AND session_dt_part = '{date}'
  AND shield_id != 'NOT_FOUND'
  AND fl.user_id IS NOT NULL
""")

shield_id.head()

Uploading - 2024-10-01: 2024-11-11 18:40:08
Uploading - 2024-10-02: 2024-11-11 18:49:44
Uploading - 2024-10-03: 2024-11-11 18:59:20
Uploading - 2024-10-04: 2024-11-11 19:08:59
Uploading - 2024-10-05: 2024-11-11 19:21:48
Uploading - 2024-10-06: 2024-11-11 19:30:23
Dataframe is empty
Uploading - 2024-10-07: 2024-11-11 19:30:42
Uploading - 2024-10-08: 2024-11-11 19:41:33
Uploading - 2024-10-09: 2024-11-11 19:55:06
Uploading - 2024-10-10: 2024-11-11 20:08:45


,user_id,platform,event_dt_part,session_dt_part,shield_id
0,222150493,iOS,2024-10-10,2024-10-10,2dd6bd5ea8764a4fe61e7912fe690037
1,57418506,iOS,2024-10-10,2024-10-10,c16595c49a8e8bb699e9829f630da1ee
2,40661590,iOS,2024-10-10,2024-10-10,0417a3458b9fd40634817918bfb2d3ea
3,134932477,iOS,2024-10-10,2024-10-10,bc6d8f5c87e72ef31f191691cde36edb
4,32634106,iOS,2024-10-10,2024-10-10,51f199f983fee14466710631e6b72343


In [41]:
shield_id.groupby(['user_id', 'platform'], as_index=False)['shield_id'].agg('nunique')['shield_id'].mean()

np.float64(1.0158029992572537)

# Writing to Excel format file

In [19]:
writing_excel("ID_Graph", active_users, new_users)

DataFrame is written to Excel File successfully.
